In [ ]:
%matplotlib notebook
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import os
import pandas as pd
import sys

sys.path.append('..')

import cogwheel
from cogwheel import parameter_estimation

In [ ]:
eventname = 'GW151226'
approximant = 'IMRPhenomXPHM'
prior_class = 'IASPrior'  # See `cogwheel.prior.prior_registry` for options
outdir = f'precessing_{eventname}'

## Set up `Posterior` object
First time compute and save to `json`. This finds a relative binning solution and takes a minute.

In [ ]:
# (Use `parameter_estimation.Posterior` instead if you don't want folding)
post = parameter_estimation.FoldedInclinationAndAzimuthPosterior.from_event(
    eventname, approximant, prior_class)

post.to_json('precessing_GW151226')

... next time you can just load this:

In [ ]:
post = parameter_estimation.read_json(
    os.path.join(outdir, f'Posterior_{eventname}.json'))

## Run PyMultinest

In [ ]:
pym = parameter_estimation.PyMultinest(post)

In [ ]:
pym.run_pymultinest(outdir, n_live_points=2000)

## Plot samples

In [ ]:
samples = pd.DataFrame(np.loadtxt(f'{outdir}/post_equal_weights.dat')[:, :-1],
                       columns=post.prior.sampled_params)

In [ ]:
sys.path.append('../../gw_detection_ias/pop_inference/')
import grid as gd

In [ ]:
gd.Grid.from_samples(list(samples), samples).corner_plot(figsize=(10, 10));